In [15]:
import pandas as pd
from glove import Corpus, Glove
corpus = Corpus() 


In [16]:
df = pd.read_csv('./df.csv') 

In [29]:
df.sort_values('label', ascending=False)[50:60]

,ref_term,content_id,label,nouns,sentences
14416,아이패드,116330469,57,아이 아이패드 패드,아이 패드
9230,에어팟 프로,116402545,56,에어 에어팟 팟 프로,에어 팟 프로
7145,jmw,116510884,56,NaN,jmw
300,아이폰xs,116534075,55,아이 아이폰 폰,아이 폰 xs
6167,kf94마스크,116455634,54,94 94마스크 마스크,kf94 마스크
31702,닌텐도 스위치,116206181,53,닌텐도 스위치,닌텐도 스위치
23247,에어팟 프로,116509840,53,에어 에어팟 팟 프로,에어 팟 프로
26948,프라이탁,115213550,53,프라이,프라이 탁
9188,클래시로얄,116535787,53,클래 클래시로얄 시 로얄,클래 시 로얄
37148,닌텐도 스위치,116533426,53,닌텐도 스위치,닌텐도 스위치


In [23]:
new_ref_term=[]
for i in df['sentences'].tolist():
    new_ref_term.append(i.split(' '))

In [37]:
df['new_ref_term'] = new_ref_term

,ref_term,content_id,label,nouns,sentences,new_ref_term
0,베이퍼맥스,116039729,3,베이 베이퍼맥스 퍼맥스,베이 퍼맥스,"[베이, 퍼맥스]"
1,솔로이스트,110951649,4,솔로 솔로이스트 이스트,솔로 이스트,"[솔로, 이스트]"
2,나이키,114052271,3,나이키,나이키,[나이키]
3,키르시 가디건,111168473,3,키 키르 르 디 디건 건,키 르시 가 디 건,"[키, 르시, 가, 디, 건]"
4,2021,116531869,4,2021,2021,[2021]
...,...,...,...,...,...,...
60170,마이멜로디 텀블러,102648097,3,마이 마이멜로디 멜로디 텀블러,마이 멜로디 텀블러,"[마이, 멜로디, 텀블러]"
60171,꼼데가르송,115365118,3,르 르송 송,꼼 데 가 르 송,"[꼼, 데, 가, 르, 송]"
60172,빗보,113869812,3,NaN,빗보,[빗보]
60173,책상 무료나눔,98036614,3,책상 무료 무료나눔 나눔,책상 무료 나눔,"[책상, 무료, 나눔]"


In [56]:
corpus.fit(new_ref_term, window=10)
glove = Glove(no_components=5, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
# glove.save('glove.model')

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [57]:
# for example
print(([0] * 5) + glove.word_vectors[glove.dictionary['베이']])

[ 0.53049134  0.05394628 -0.92614803 -0.58455545 -0.98691351]


In [58]:
glove_list = []
for term in new_ref_term:
    value = [0] * 5
    for i in term:
        value += glove.word_vectors[glove.dictionary[i]]
    glove_list.append(value)

In [61]:
one = []
two = []
three = []
four = []
five= []
for i in glove_list:
    one.append(i[0])
    two.append(i[1])
    three.append(i[2])
    four.append(i[3])
    five.append(i[4])

In [62]:
df['one'] = one
df['two'] = two
df['three'] = three
df['four'] = four
df['five'] = five

,ref_term,content_id,label,nouns,sentences,new_ref_term,one,two,three,four,five
0,베이퍼맥스,116039729,3,베이 베이퍼맥스 퍼맥스,베이 퍼맥스,"[베이, 퍼맥스]",0.997243,-0.103179,-1.640525,-1.133015,-1.739105
1,솔로이스트,110951649,4,솔로 솔로이스트 이스트,솔로 이스트,"[솔로, 이스트]",-0.573198,-0.022832,0.302212,0.443954,-0.985549
2,나이키,114052271,3,나이키,나이키,[나이키],-0.589855,-0.893785,-1.167207,0.135240,-0.437617
3,키르시 가디건,111168473,3,키 키르 르 디 디건 건,키 르시 가 디 건,"[키, 르시, 가, 디, 건]",-1.726663,-2.662513,2.937025,1.194998,3.532659
4,2021,116531869,4,2021,2021,[2021],0.031961,0.116597,-0.037542,0.117307,0.027203


In [113]:
target_cols = ['one', 'two', 'three', 'four', 'five', 'ref_term', 'content_id']
label_dict = {}
for col in target_cols:
    unique = df[col].unique()
    row_number = range(len(unique))
    label = pd.DataFrame.from_dict({col: unique, 'label': row_number})
    label[col] = label[col].astype(str)
    label_dict[col] = label
    label.to_csv('./' + col + '_label.csv', index=False, header=False)

In [114]:
x = df[target_cols + ['label']]
for col in x.columns:
    x[col] = x[col].astype(str)
y1 = label_dict['one']
y2 = label_dict['two']
y3 = label_dict['three']
y4 = label_dict['four']
y5 = label_dict['five']
y6 = label_dict['content_id']
y7 = label_dict['ref_term']

/Users/emilyPark/.pyenv/versions/3.6.9/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [116]:
keyword_label = pd.merge(x, y1, on='one')
keyword_label = pd.merge(keyword_label, y2, on='two')
keyword_label = pd.merge(keyword_label, y3, on='three')
keyword_label = pd.merge(keyword_label, y4, on='four')
keyword_label = pd.merge(keyword_label, y5, on='five')
keyword_label = pd.merge(keyword_label, y6, on='content_id')
keyword_label = pd.merge(keyword_label, y7, on='ref_term')
keyword_label
keyword_label.columns = ['one', 'two', 'three', 'four', 'five', 'ref_term', 'content_id', 'label', 'one_label', 'two_label', 'three_label', 'four_label', 'five_label', 'content_id_label', 'ref_term_label']

In [119]:
one_max = keyword_label['one_label'].max()
two_max = keyword_label['two_label'].max()
three_max = keyword_label['three_label'].max()
four_max = keyword_label['four_label'].max()
five_max = keyword_label['five_label'].max()
content_max = keyword_label['content_id_label'].max()
ref_term_max = keyword_label['ref_term_label'].max()

In [89]:
# keyword_label.to_csv('./keyword_label.csv', index=False)

In [120]:
for col in keyword_label.columns:
    keyword_label[col] = keyword_label[col].astype(str)

In [110]:
libsvm = []
maxs = [one_max, two_max, three_max, four_max, five_max, ]
for i in range(len(keyword_label)):
    r = keyword_label.iloc[i]
    s = str(r['label']) + ' ' + str(r['one_label']) + ':1'
    index = 0
    mx = 0
    for j in ['two_label', 'three_label', 'four_label', 'five_label', 'content_id_label']:
        s += ' '
        mx += maxs[index]
        index += 1
        s = s + str(int(r[j]) + mx) + ':1'
    libsvm.append(s)
libsvm_csv = pd.DataFrame.from_dict({'libsvm': libsvm})
libsvm_csv.to_csv('./libsvm.csv', index=False, header=False)